In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [3]:
data = pd.read_csv('/content/drive/MyDrive/DL notebooks/titanic.csv')


In [4]:

x = data[['Pclass', 'Age', 'Sex']].copy()
y = data[['Survived']].values.flatten()

x['n_age'] = x['Age'].fillna(x['Age'].mean())
x['N_Sex'] = x['Sex'].apply(lambda x: 1 if x == 'male' else 0)
x.drop(["Sex", "Age"], axis=1, inplace=True)

scaler = StandardScaler()
x = scaler.fit_transform(x)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [6]:

import math
def sigmoid(x):
  return 1 / (1+math.exp(-x))


In [7]:

def sigmoid_numpy(x):
  return 1 / (1+np.exp(-x))

sigmoid_numpy(np.array([-0.369365,-1.823750,-1.355574]))

array([0.40869447, 0.13898451, 0.20496058])

In [8]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = np.clip(y_predicted, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))


In [25]:
class NN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.w3 = 1
        self.bias = 0

    def fit(self, x, y, epochs, loss_threshold):
        # Adapted for NumPy arrays: access columns by index
        self.w1, self.w2, self.w3, self.bias = self.gradient_descent(
            x[:, 0], x[:, 1], x[:, 2], y, epochs, loss_threshold
        )
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, w3: {self.w3}, bias: {self.bias}")

    def predict(self, x_test):
        # Adapted for NumPy arrays: access columns by index
        weighted_sum = self.w1 * x_test[:, 0] + self.w2 * x_test[:, 1] + self.w3 * x_test[:, 2] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, Pclass, Age, Sex, y_true, epochs, loss_threshold):
        w1 = w2 = w3 = 1
        bias = 0
        rate = 0.5
        n = len(Pclass)

        for i in range(epochs):
            weighted_sum = w1 * Pclass + w2 * Age + w3 * Sex + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            # Calculate gradients
            w1d = (1 / n) * np.dot(Pclass, (y_predicted - y_true))
            w2d = (1 / n) * np.dot(Age, (y_predicted - y_true))
            w3d = (1 / n) * np.dot(Sex, (y_predicted - y_true))
            bias_d = np.mean(y_predicted - y_true)

            # Update weights and bias
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            w3 = w3 - rate * w3d
            bias = bias - rate * bias_d

            # Log progress
            if i % 50 == 0:  # Print every 50 epochs
                print(f"Epoch {i + 1}: w1={w1:.4f}, w2={w2:.4f}, w3={w3:.4f}, bias={bias:.4f}, loss={loss:.4f}")

            # Stop if loss reaches threshold
            if loss <= loss_threshold:
                print(f"Converged at Epoch {i + 1}: w1={w1:.4f}, w2={w2:.4f}, w3={w3:.4f}, bias={bias:.4f}, loss={loss:.4f}")
                break

        return w1, w2, w3, bias


In [26]:
craftModel = NN()
craftModel.fit(x_train, y_train, epochs=100, loss_threshold=0.4631)

Epoch 1: w1=0.8512, w2=0.9246, w3=0.7592, bias=-0.0679, loss=1.4248
Converged at Epoch 43: w1=-0.7684, w2=-0.2420, w3=-1.1550, bias=-0.5679, loss=0.4629
Final weights and bias: w1: -0.768383622000202, w2: -0.24197343043768987, w3: -1.155009052199299, bias: -0.5678548970942839
